# Langchain Expression Language (LCEL)

**With Runnable interface creating custom chain becomes easy. Langchains components like LLMs, retriever, output_parser, etc follows Runnable.**

In [19]:
# !pip install -U langchain-google-genai
# !where python
# !pip install langchain 

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda, Runnable
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key="")

C:\Users\pooja\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Pipe operator: Sequential Chaining

**With Pipe operator, two runnables can be chained together sequentially. The resulting sequence is also a runnable component.**

**The output of previous runnable component is passed as input to next component.**

In [9]:
# create a chain to translate content to english.
prompt = PromptTemplate.from_template("Translate the {text} to english")

# define chain using LLMchain
chain = LLMChain(llm=llm, prompt=prompt)

# define chain using pipe operator
pipe_chain = prompt | llm 

In [15]:
text = """
Generative KI oder generative künstliche Intelligenz bezieht sich auf die Verwendung von KI zum Erstellen neuer Inhalte wie Texte, Bilder, Musik, Audio und Videos.
"""

output = chain.invoke({"text": text})
print('Output of chain using LLMChain function: \n', output)

Output of chain using LLMChain function: 
 {'text': 'Generative AI or generative artificial intelligence refers to the use of AI to create new content such as text, images, music, audio and video. \n'}


In [12]:
pipe_output = pipe_chain.invoke({"text": text})
print('Output of chain using pipe operator: \n', pipe_output.content)

Output of chain using pipe operator: 
 Generative AI, or generative artificial intelligence, refers to the use of AI to create new content such as text, images, music, audio, and videos. 



## RunnableLambda

**To convert custom functions to runnable compatible RunnableLambda is used.**

In [70]:
template = """
Detect language of the {text}. Translate it to english.
{format_instructions}
"""

class language(BaseModel):
    language: str = Field(description="Detected language")
    translation: str = Field(description="English tranlation")   

**The following chain will output language and translation based on pydantic object. But final output is altered to translation with custom python function. This function is converted to runnable compatible with RunnableLambda.** 

In [71]:
parser = JsonOutputParser(pydantic_object=language)
prompt = PromptTemplate(template=template,
                        input_varibles=['text'],
                       partial_variables={'format_instructions': parser.get_format_instructions()})
lambda_chain = prompt | llm | parser | RunnableLambda(lambda x: x['translation'])

In [72]:
content = """
Generative KI oder generative künstliche Intelligenz bezieht sich auf die Verwendung von KI zum Erstellen neuer Inhalte wie Texte, Bilder, Musik, Audio und Videos.
"""

lambda_output = lambda_chain.invoke({"text": content})

In [74]:
print("Runnablelamda altered output: \n", lambda_output)

Runnablelamda altered output: 
 Generative AI or generative artificial intelligence refers to the use of AI to create new content such as text, images, music, audio and videos.


## RunnableParallel

**As name suggests, the RunnableParallel helps to execute 2 chains in parallel.**

**The first chain detects language and other translates the given content. The output of RunnableParallel contains language and translation.**

In [75]:
template1 = """
Detect language of the {text}.
"""
prompt1 = PromptTemplate.from_template(template1)
chain1 = prompt1 | llm

In [76]:
template2 = """
Translate the {text} to English.
"""
prompt2 = PromptTemplate.from_template(template2)
chain2 = prompt2 | llm

In [80]:
content = """
Generative KI oder generative künstliche Intelligenz bezieht sich auf die Verwendung von KI zum Erstellen neuer Inhalte wie Texte, Bilder, Musik, Audio und Videos.
"""

parallel_chain = RunnableParallel(translate=chain2, Language=chain1)
parallel_output = parallel_chain.invoke({"text": content})

In [86]:
print('RunnableParallel output: \n')
print('translation: \n', parallel_output['translate'])
print('*' * 100)
print('language: \n', parallel_output['Language'])

RunnableParallel output: 

translation: 
 content='Generative AI, or generative artificial intelligence, refers to the use of AI to create new content such as text, images, music, audio, and video. \n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-f74268b4-c06c-4153-bb53-66977cd36514-0' usage_metadata={'input_tokens': 43, 'output_tokens': 32, 'total_tokens': 75}
****************************************************************************************************
language: 
 content='The language is **German**. \n\nT

## RunnablePassthrough

**RunnablePassthrough allows to pass values without any change.**

In [2]:
template = """
Translate the {text} to English.
"""
prompt = PromptTemplate.from_template(template)
chain = prompt | llm

**The text varible passes original content a it is with translate varible containing chain output in parallel.**

In [4]:
content = """
Generative KI oder generative künstliche Intelligenz bezieht sich auf die Verwendung von KI zum Erstellen neuer Inhalte wie Texte, Bilder, Musik, Audio und Videos.
"""

pass_chain = RunnableParallel(translate=chain, text=RunnablePassthrough())
pass_output = pass_chain.invoke({"text": content})

In [6]:
print('RunnablePassthrough output: \n', pass_output)
print("*"*100)
print('Passthrough value: ', pass_output['text'])
print("*"*100)
print('translation output: ', pass_output['translate'])

RunnablePassthrough output: 
 {'translate': AIMessage(content='Generative AI, or generative artificial intelligence, refers to the use of AI to create new content such as text, images, music, audio, and video. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-38b78e0d-75f6-4c15-87b1-c5c703ec8c63-0', usage_metadata={'input_tokens': 43, 'output_tokens': 32, 'total_tokens': 75}), 'text': {'text': '\nGenerative KI oder generative künstliche Intelligenz bezieht sich auf die Verwendung von KI zum Erstellen neuer Inhal

## RunnablePassthrough.assign

**The assign method add extra variable to the current chain output.**

In [22]:
template2 = """
Translate the {text} to English.
"""
prompt2 = PromptTemplate.from_template(template2)
chain2 = prompt2 | llm

In [23]:
content = """
Generative KI oder generative künstliche Intelligenz bezieht sich auf die Verwendung von KI zum Erstellen neuer Inhalte wie Texte, Bilder, Musik, Audio und Videos.
"""

assign_chain = RunnablePassthrough.assign(output=chain2)
assign_output = assign_chain.invoke({"text": content})

In [24]:
print("Output of assign: \n", assign_output)

Output of assign: 
 {'text': '\nGenerative KI oder generative künstliche Intelligenz bezieht sich auf die Verwendung von KI zum Erstellen neuer Inhalte wie Texte, Bilder, Musik, Audio und Videos.\n', 'output': AIMessage(content='Generative AI, or generative artificial intelligence, refers to the use of AI to create new content such as text, images, music, audio, and videos. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a065d971-d58b-4bda-88e5-cbbf07488ce1-0', usage_metadata={'input_tokens': 43, 'output_token